In [6]:
import pandas as pd
import re

In [7]:
# Text Cleaning and Regular Expression
def regex(text):
    """
    Applies regular expression to a text to remove punctuation marks
    """
    text = re.sub(r'[^\w\s]', "", text) # Replace punctuation marks with empty string
    text = re.sub(r'[\s+]', " ", text) # Replace multiple spaces with one space
    
    return text.strip()
    
def clean_text(dataframe, text_col):
    """
    Ensures data is consistent and removes punctuation for better model performance.
    """
    df = pd.read_csv(dataframe) # Initialize DataFrame
    
    df = df.drop(df.duplicated) # Remove Duplicated rows
    df = df.dropna(subset=[text_col]) # Remove rows with missing values in text column
    
    df[text_col] = df[text_col].apply(regex) # Remove punctuation marks
    
    return df

In [ ]:
# Tokenizaton

In [ ]:
# Stop Word Removal

In [ ]:
# Lemmatization

In [ ]:
# Vectorization